Encoding techiniques for categorical data are compulsary to understand this.  
Look at these notebooks:  
* [OrdinalEncoding](./HandlingCategoricalFeatures/EncodingTechiniques/1.%20ordinalEncoding.ipynb)  
* [OneHotEncoding](./HandlingCategoricalFeatures/EncodingTechiniques/2.%20OneHotEncoding.ipynb)


# **Column Transformer**  
Do all of the hectic encoding stuff in one line... no need to separate columns and then re-stack them.

In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [6]:
df = pd.read_csv("assets/covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


### Plan  

* gender (nominal categorical) : OneHotEncoder  
* city (nominal categorical) : OneHotEncoder  
* cough (ordinal categorical) : OrdinalEncoder
* fever (numeric, has nulls) : SimpleImputer

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop('has_covid',axis=1),
                                                df['has_covid'],
                                                test_size=0.2)

### Without Column Trnaformer:

In [29]:
si = SimpleImputer()

si.fit(X_train[['fever']])
X_train_fever = si.transform(X_train[['fever']])
X_test_fever = si.transform(X_test[['fever']])

In [30]:
X_train_fever.shape,X_test_fever.shape

((80, 1), (20, 1))

In [35]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough  = oe.transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [38]:
oh = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = oh.fit_transform(X_train[['gender','city']])
X_test_gender_city = oh.transform(X_test[['gender','city']])
X_train_gender_city.shape

(80, 4)

In [45]:
X_test.columns

Index(['age', 'gender', 'fever', 'cough', 'city'], dtype='object')

In [46]:
#get age to add back to all these transformed columns

X_train_age = X_train.drop(X_train.columns[1:5],axis=1)
X_test_age = X_test.drop(X_test.columns[1:5],axis=1)
X_train_age.columns,X_test_age.columns

(Index(['age'], dtype='object'), Index(['age'], dtype='object'))

In [51]:
X_train_transformed = np.concatenate((X_train_age,X_train_gender_city,X_train_cough,X_train_fever),axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_gender_city,X_test_cough,X_test_fever),axis=1)
X_train_transformed.shape,X_test_transformed.shape

((80, 7), (20, 7))

In [56]:
X_train_transformed

array([[ 49.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        , 101.        ],
       [ 23.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        , 100.76388889],
       [ 12.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  98.        ],
       [ 11.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   1.        , 100.        ],
       [ 47.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        , 100.        ],
       [ 69.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   1.        ,  98.        ],
       [  6.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        , 104.        ],
       [  5.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        , 100.        ],
       [ 83.        ,   1.        ,   1.        ,   0.        ,
          0.    

## With Column Transformer

In [52]:
from sklearn.compose import ColumnTransformer

In [54]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])
],remainder='passthrough')

In [55]:
transformer.fit_transform(X_train)

array([[101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [100.76388889,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  23.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  12.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  11.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  47.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  69.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   6.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [ 98.        ,   0.        ,   1.        ,   1.        ,
          0.    

In [57]:
transformer.transform(X_test)

array([[ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  72.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  19.        ],
       [103.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  70.        ],
       [ 99.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  66.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  69.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  10.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  16.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          0.    